In [1]:
import pandas as pd
from sklearn import *
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import LabelEncoder


In [2]:
path = '/Users/rosreen/Desktop/oracle-dbt/seeds/'
file = path + 'invoices.csv'

data = pd.read_csv(file)
data.head()

/var/folders/8v/s5jwzy2568j12y51zbwc4b8w0000gq/T/ipykernel_91723/1117076995.py:4: DtypeWarning: Columns (50) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(file)


,Unnamed: 0,SOURCE_RECORD_ID,INVOICE_ROUTING_ATTR_3,PAYMENT_REASON_COMMENTS,INVOICE_CANCELLED_FLAG,SECOND_DISCOUNT_LEDGER_AMOUNT,INVOICE_VALIDATION_STATUS_CODE,REMAINING_GLOBAL_AMOUNT,PRE_POST_SLA_FLAG,HEADER_UPDATED_BY_USER_ID,...,INVOICED_ON_DATE,INVOICE_RECEIVED_DATE,INVOICE_SCHEDULE_DUE_DATE,FIRST_DISCOUNT_DATE,SECOND_DISCOUNT_DATE,THIRD_DISCOUNT_DATE,INVOICE_CREATION_DATE_1,INVOICE_ACCOUNTING_DATE,INVC_SCHEDULE_CLEARED_DATE,INVOICE_CREATION_DATE
0,0,366287~1,NaN,NaN,NO,0.0,APPROVED,636.493600,PRE,AP_SUPV_PD,...,11/16/18,1/1/01,11/16/18,1/1/01,1/1/01,1/1/01,11/16/18,11/16/18,53:00.0,18:48.0
1,1,374072~1,NaN,NaN,NO,0.0,APPROVED,0.000000,PRE,AP_SUPV_PD,...,11/19/18,1/1/01,11/19/18,1/1/01,1/1/01,1/1/01,11/19/18,11/19/18,59:55.0,50:46.0
2,2,368123~1,NaN,NaN,NO,0.0,APPROVED,3279.814080,PRE,AP_SUPV_PD,...,11/19/18,1/1/01,11/19/18,1/1/01,1/1/01,1/1/01,11/19/18,11/19/18,53:00.0,21:08.0
3,3,248831~1,NaN,NaN,NO,0.0,APPROVED,3494.252874,POST,AP_SUPV_ITALY,...,11/20/06,1/1/01,11/20/06,1/1/01,1/1/01,1/1/01,12/8/15,11/20/06,53:00.0,07:33.0
4,4,366184~1,NaN,NaN,NO,0.0,APPROVED,3624.269440,PRE,AP_SUPV_PD,...,11/16/18,1/1/01,11/16/18,1/1/01,1/1/01,1/1/01,11/16/18,11/16/18,53:00.0,36:57.0


In [3]:
df = data.dropna(axis = 1, how = 'any')
df = df.dropna(axis = 0, how = 'all')
data = df.apply(LabelEncoder().fit_transform)
data = data.loc[:, (df != 0).any(axis=0)]
data.head()

,Unnamed: 0,SOURCE_RECORD_ID,INVOICE_CANCELLED_FLAG,SECOND_DISCOUNT_LEDGER_AMOUNT,INVOICE_VALIDATION_STATUS_CODE,PRE_POST_SLA_FLAG,HEADER_UPDATED_BY_USER_ID,THIRD_PARTY_ID,HEADER_CREATED_BY_USER_ID,PAYMENT_TERMS_ID,...,INVOICED_ON_DATE,INVOICE_RECEIVED_DATE,INVOICE_SCHEDULE_DUE_DATE,FIRST_DISCOUNT_DATE,SECOND_DISCOUNT_DATE,THIRD_DISCOUNT_DATE,INVOICE_CREATION_DATE_1,INVOICE_ACCOUNTING_DATE,INVC_SCHEDULE_CLEARED_DATE,INVOICE_CREATION_DATE
0,0,12309,0,0,0,1,21,0,18,0,...,254,0,267,0,0,0,51,226,628,571
1,1,17294,0,0,0,1,21,0,18,0,...,258,0,274,0,0,0,53,230,713,1605
2,2,12815,0,0,0,1,21,0,18,0,...,258,0,274,0,0,0,53,230,628,644
3,3,2507,0,0,0,0,17,0,14,0,...,260,0,276,0,0,0,86,232,628,241
4,4,12206,0,0,0,1,21,0,18,0,...,254,0,267,0,0,0,51,226,628,1140


In [4]:
features = data.columns

# Train the Isolation Forest model
model = IsolationForest(n_estimators=100, contamination='auto')
model.fit(data[features])

# Predict the anomalies
anomalies = model.predict(data[features])

# Add the anomalies to the original data
data['anomaly'] = anomalies

# Print the anomalies
res_df = data[data['anomaly'] == -1]
res_df


,Unnamed: 0,SOURCE_RECORD_ID,INVOICE_CANCELLED_FLAG,SECOND_DISCOUNT_LEDGER_AMOUNT,INVOICE_VALIDATION_STATUS_CODE,PRE_POST_SLA_FLAG,HEADER_UPDATED_BY_USER_ID,THIRD_PARTY_ID,HEADER_CREATED_BY_USER_ID,PAYMENT_TERMS_ID,...,INVOICE_RECEIVED_DATE,INVOICE_SCHEDULE_DUE_DATE,FIRST_DISCOUNT_DATE,SECOND_DISCOUNT_DATE,THIRD_DISCOUNT_DATE,INVOICE_CREATION_DATE_1,INVOICE_ACCOUNTING_DATE,INVC_SCHEDULE_CLEARED_DATE,INVOICE_CREATION_DATE,anomaly
13,13,3099,0,0,0,0,81,0,84,0,...,0,308,0,0,0,64,172,628,592,-1
16,16,7230,0,0,5,1,2,0,0,0,...,0,70,0,0,0,7,63,628,310,-1
19,19,10696,0,0,0,0,18,0,38,0,...,0,472,0,0,0,168,392,523,932,-1
22,22,20334,0,0,4,1,31,0,28,0,...,0,941,0,0,0,174,785,628,918,-1
23,23,21051,0,0,4,1,31,0,28,0,...,0,941,0,0,0,174,785,628,918,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22433,22433,6622,0,0,0,0,49,0,51,0,...,0,630,0,0,0,257,640,628,1694,-1
22435,22435,6519,0,0,0,1,2,0,48,0,...,0,1132,0,0,0,234,904,628,1436,-1
22436,22436,19117,0,11,4,1,2,0,75,15,...,0,1120,257,36,35,264,969,628,572,-1
22437,22437,7066,0,0,0,0,2,0,1,1,...,0,290,0,0,0,47,196,628,812,-1
